In [ ]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from skfuzzy import control as ctrl
import pickle
import json

In [ ]:
# Open the 'info' file, where the pickled data is stored
file = open('info', 'rb')

# Load information from that file
scraped_info = pickle.load(file)

# Close the file
file.close()

# Store scraped info in new variables according to different categories
weather_data = scraped_info['weather_data']
month_data = scraped_info['month_data']

In [ ]:
'''
index 0  = kota_kinabalu
index 1  = tawau
index 2  = sandakan
index 3  = kuching
index 4  = miri
index 5  = sibu
index 6  = kota_bharu
index 7  = kuantan
index 8  = kuala_terengganu
index 9  = kangar
index 10 = alor_setar
index 11 = georgetown
index 12 = ipoh
index 13 = shah_alam
index 14 = kuala_lumpur
index 15 = putrajaya
index 16 = seremban
index 17 = malacca
index 18 = johor_bahru
'''

In [ ]:
# List of city names
destination_list = [
"Kota-Kinabalu",
"Tawau",
"Sandakan",
"Kuching",
"Miri",
"Sibu",
"Kota-Bharu",
"Kuantan",
"Kuala-Terengganu",
"Kangar",
"Alor-Setar",
"George-Town",
"Ipoh",
"Shah-Alam",
"Kuala-Lumpur",
"Putrajaya",
"Seremban",
"Malacca",
"Johor-Bahru",
]

In [ ]:
# Retrieve average weather information
lowest_temp = weather_data['lowest_temp']
lowest_clear = weather_data['lowest_clear']
lowest_precipitation = weather_data['lowest_precipitation']
lowest_rainfall = weather_data['lowest_rainfall']
lowest_wind = weather_data['lowest_wind']
avg_temp = weather_data['avg_temp']
avg_clear = weather_data['avg_clear']
avg_precipitation = weather_data['avg_precipitation']
avg_rainfall = weather_data['avg_rainfall']
avg_wind = weather_data['avg_wind']
highest_temp = weather_data['highest_temp']
highest_clear = weather_data['highest_clear']
highest_precipitation = weather_data['highest_precipitation']
highest_rainfall = weather_data['highest_rainfall']
highest_wind = weather_data['highest_wind']

In [ ]:
# Retrieve month information
jan_info = month_data['JAN']
feb_info = month_data['FEB']
mar_info = month_data['MAR']
apr_info = month_data['APR']
may_info = month_data['MAY']
jun_info = month_data['JUN']
jul_info = month_data['JUL']
aug_info = month_data['AUG']
sep_info = month_data['SEP']
oct_info = month_data['OCT']
nov_info = month_data['NOV']
dec_info = month_data['DEC']

In [ ]:
def build_mb_function(index):
    # New Antecedent/Consequent objects hold universe variables and membership
    # functions
    precipitation = ctrl.Antecedent(np.arange(lowest_precipitation[index], highest_precipitation[index]+0.05, 0.05), 'precipitation')
    clear = ctrl.Antecedent(np.arange(lowest_clear[index], highest_clear[index]+0.05, 0.05), 'clear')
    result = ctrl.Consequent(np.arange(0, 110, 1), 'result')

    # Custom membership functions can be built interactively with a familiar,
    # Pythonic API
    precipitation['low'] = fuzz.trimf(precipitation.universe, [lowest_precipitation[index], lowest_precipitation[index], avg_precipitation[index]])
    precipitation['medium'] = fuzz.trimf(precipitation.universe, [lowest_precipitation[index], avg_precipitation[index], highest_precipitation[index]])
    precipitation['high'] = fuzz.trimf(precipitation.universe, [avg_precipitation[index], highest_precipitation[index], highest_precipitation[index]])

    clear['low'] = fuzz.trimf(clear.universe, [lowest_clear[index], lowest_clear[index], avg_clear[index]])
    clear['medium'] = fuzz.trimf(clear.universe, [lowest_clear[index], avg_clear[index], highest_clear[index]])
    clear['high'] = fuzz.trimf(clear.universe, [avg_clear[index], highest_clear[index], highest_clear[index]])

    result['low'] = fuzz.trimf(result.universe, [0, 0, 50])
    result['high'] = fuzz.trimf(result.universe, [50, 100, 100])

    return (precipitation, clear, result)


In [ ]:
'''
build_mb_function()[]
---------------------
() -> destination (city name)
[] -> 0 = precipitation, 1 = cloud, 2 = result
'''

In [ ]:
def Rules(index):
    a_precipitation = build_mb_function(index)[0]
    a_clear = build_mb_function(index)[1]
    c_result = build_mb_function(index)[2]

    # Fuzzy rules
    rule1 = ctrl.Rule(a_clear['high'] | a_precipitation['low'], c_result['high'])
    rule2 = ctrl.Rule(a_clear['low'] | a_precipitation['high'], c_result['low'])

    # Control System Creation and Simulation
    tipping_ctrl = ctrl.ControlSystem([rule1, rule2])
    tipping = ctrl.ControlSystemSimulation(tipping_ctrl)

    return tipping


In [ ]:
def Compute_result(rules_tipping, month):
    # Pass inputs to the ControlSystem using Antecedent labels with Pythonic API
    # Note: if you like passing many inputs all at once, use .inputs(dict_of_data)
    rules_tipping.input['clear'] = month[4]
    rules_tipping.input['precipitation'] = month[5]

    # Crunch the numbers
    rules_tipping.compute()

    return rules_tipping.output['result']

In [ ]:
jan_prob_list = []
feb_prob_list = []
mar_prob_list = []
apr_prob_list = []
may_prob_list = []
jun_prob_list = []
jul_prob_list = []
aug_prob_list = []
sep_prob_list = []
oct_prob_list = []
nov_prob_list = []
dec_prob_list = []

# Compute the result for each destination/city
for index in range(len(destination_list)):
    jan_prob = Compute_result(Rules(index), jan_info[index])
    feb_prob = Compute_result(Rules(index), feb_info[index])
    mar_prob = Compute_result(Rules(index), mar_info[index])
    apr_prob = Compute_result(Rules(index), apr_info[index])
    may_prob = Compute_result(Rules(index), may_info[index])
    jun_prob = Compute_result(Rules(index), jun_info[index])
    jul_prob = Compute_result(Rules(index), jul_info[index])
    aug_prob = Compute_result(Rules(index), aug_info[index])
    sep_prob = Compute_result(Rules(index), sep_info[index])
    oct_prob = Compute_result(Rules(index), oct_info[index])
    nov_prob = Compute_result(Rules(index), nov_info[index])
    dec_prob = Compute_result(Rules(index), dec_info[index])

    jan_prob_list.append(jan_prob)
    feb_prob_list.append(feb_prob)
    mar_prob_list.append(mar_prob)
    apr_prob_list.append(apr_prob)
    may_prob_list.append(may_prob)
    jun_prob_list.append(jun_prob)
    jul_prob_list.append(jul_prob)
    aug_prob_list.append(aug_prob)
    sep_prob_list.append(sep_prob)
    oct_prob_list.append(oct_prob)
    nov_prob_list.append(nov_prob)
    dec_prob_list.append(dec_prob)

In [ ]:
'''
Example: jan_info[index x][index y]
index x -> 0-18 = index for destinations/cities

index y -> 1 = Temperature
index y -> 4 = Cloud (Clearer sky)
index y -> 5 = Rain days (Precipitation)
index y -> 6 = Rainfall
index y -> 9 = Wind Speed
'''

In [ ]:
# Setting up format for the JSON file
weather_intents = {
    "intent_dict" : [
        {
            "destination" : "Kota Kinabalu",
            "months" :{
                "january" :{
                    "probability" : jan_prob_list[0],
                    "avg_temp" : jan_info[0][1],
                    "clearer" : jan_info[0][4],
                    "precipitation" : jan_info[0][5],
                    "rainfall" : jan_info[0][6],
                    "windspeed" : jan_info[0][9]
                },
                "february" :{
                    "probability" : jan_prob_list[0],
                    "avg_temp" : feb_info[0][1],
                    "clearer" : feb_info[0][4],
                    "precipitation" : feb_info[0][5],
                    "rainfall" : feb_info[0][6],
                    "windspeed" : feb_info[0][9]
                },
                "march" :{
                    "probability" : mar_prob_list[0],
                    "avg_temp" : mar_info[0][1],
                    "clearer" : mar_info[0][4],
                    "precipitation" : mar_info[0][5],
                    "rainfall" : mar_info[0][6],
                    "windspeed" : mar_info[0][9]
                },
                "april" :{
                    "probability" : apr_prob_list[0],
                    "avg_temp" : apr_info[0][1],
                    "clearer" : apr_info[0][4],
                    "precipitation" : apr_info[0][5],
                    "rainfall" : apr_info[0][6],
                    "windspeed" : apr_info[0][9]
                },
                "may" :{
                    "probability" : may_prob_list[0],
                    "avg_temp" : may_info[0][1],
                    "clearer" : may_info[0][4],
                    "precipitation" : may_info[0][5],
                    "rainfall" : may_info[0][6],
                    "windspeed" : may_info[0][9]
                },
                "june" :{
                    "probability" : jun_prob_list[0],
                    "avg_temp" : jun_info[0][1],
                    "clearer" : jun_info[0][4],
                    "precipitation" : jun_info[0][5],
                    "rainfall" : jun_info[0][6],
                    "windspeed" : jun_info[0][9]
                },
                "july" :{
                    "probability" : jul_prob_list[0],
                    "avg_temp" : jul_info[0][1],
                    "clearer" : jul_info[0][4],
                    "precipitation" : jul_info[0][5],
                    "rainfall" : jul_info[0][6],
                    "windspeed" : jul_info[0][9]
                },
                "august" :{
                    "probability" : aug_prob_list[0],
                    "avg_temp" : aug_info[0][1],
                    "clearer" : aug_info[0][4],
                    "precipitation" : aug_info[0][5],
                    "rainfall" : aug_info[0][6],
                    "windspeed" : aug_info[0][9]
                },
                "september" :{
                    "probability" : sep_prob_list[0],
                    "avg_temp" : sep_info[0][1],
                    "clearer" : sep_info[0][4],
                    "precipitation" : sep_info[0][5],
                    "rainfall" : sep_info[0][6],
                    "windspeed" : sep_info[0][9]
                },
                "october" :{
                    "probability" : oct_prob_list[0],
                    "avg_temp" : oct_info[0][1],
                    "clearer" : oct_info[0][4],
                    "precipitation" : oct_info[0][5],
                    "rainfall" : oct_info[0][6],
                    "windspeed" : oct_info[0][9]
                },
                "november" :{
                    "probability" : nov_prob_list[0],
                    "avg_temp" : nov_info[0][1],
                    "clearer" : nov_info[0][4],
                    "precipitation" : nov_info[0][5],
                    "rainfall" : nov_info[0][6],
                    "windspeed" : nov_info[0][9]
                },
                "december" :{
                    "probability" : dec_prob_list[0],
                    "avg_temp" : dec_info[0][1],
                    "clearer" : dec_info[0][4],
                    "precipitation" : dec_info[0][5],
                    "rainfall" : dec_info[0][6],
                    "windspeed" : dec_info[0][9]
                }
            }
        },
        {
            "destination" : "Tawau",
            "months" :{
                "january" :{
                    "probability" : jan_prob_list[1],
                    "avg_temp" : jan_info[1][1],
                    "clearer" : jan_info[1][4],
                    "precipitation" : jan_info[1][5],
                    "rainfall" : jan_info[1][6],
                    "windspeed" : jan_info[1][9]
                },
                "february" :{
                    "probability" : jan_prob_list[1],
                    "avg_temp" : feb_info[1][1],
                    "clearer" : feb_info[1][4],
                    "precipitation" : feb_info[1][5],
                    "rainfall" : feb_info[1][6],
                    "windspeed" : feb_info[1][9]
                },
                "march" :{
                    "probability" : mar_prob_list[1],
                    "avg_temp" : mar_info[1][1],
                    "clearer" : mar_info[1][4],
                    "precipitation" : mar_info[1][5],
                    "rainfall" : mar_info[1][6],
                    "windspeed" : mar_info[1][9]
                },
                "april" :{
                    "probability" : apr_prob_list[1],
                    "avg_temp" : apr_info[1][1],
                    "clearer" : apr_info[1][4],
                    "precipitation" : apr_info[1][5],
                    "rainfall" : apr_info[1][6],
                    "windspeed" : apr_info[1][9]
                },
                "may" :{
                    "probability" : may_prob_list[1],
                    "avg_temp" : may_info[1][1],
                    "clearer" : may_info[1][4],
                    "precipitation" : may_info[1][5],
                    "rainfall" : may_info[1][6],
                    "windspeed" : may_info[1][9]
                },
                "june" :{
                    "probability" : jun_prob_list[1],
                    "avg_temp" : jun_info[1][1],
                    "clearer" : jun_info[1][4],
                    "precipitation" : jun_info[1][5],
                    "rainfall" : jun_info[1][6],
                    "windspeed" : jun_info[1][9]
                },
                "july" :{
                    "probability" : jul_prob_list[1],
                    "avg_temp" : jul_info[1][1],
                    "clearer" : jul_info[1][4],
                    "precipitation" : jul_info[1][5],
                    "rainfall" : jul_info[1][6],
                    "windspeed" : jul_info[1][9]
                },
                "august" :{
                    "probability" : aug_prob_list[1],
                    "avg_temp" : aug_info[1][1],
                    "clearer" : aug_info[1][4],
                    "precipitation" : aug_info[1][5],
                    "rainfall" : aug_info[1][6],
                    "windspeed" : aug_info[1][9]
                },
                "september" :{
                    "probability" : sep_prob_list[1],
                    "avg_temp" : sep_info[1][1],
                    "clearer" : sep_info[1][4],
                    "precipitation" : sep_info[1][5],
                    "rainfall" : sep_info[1][6],
                    "windspeed" : sep_info[1][9]
                },
                "october" :{
                    "probability" : oct_prob_list[1],
                    "avg_temp" : oct_info[1][1],
                    "clearer" : oct_info[1][4],
                    "precipitation" : oct_info[1][5],
                    "rainfall" : oct_info[1][6],
                    "windspeed" : oct_info[1][9]
                },
                "november" :{
                    "probability" : nov_prob_list[1],
                    "avg_temp" : nov_info[1][1],
                    "clearer" : nov_info[1][4],
                    "precipitation" : nov_info[1][5],
                    "rainfall" : nov_info[1][6],
                    "windspeed" : nov_info[1][9]
                },
                "december" :{
                    "probability" : dec_prob_list[1],
                    "avg_temp" : dec_info[1][1],
                    "clearer" : dec_info[1][4],
                    "precipitation" : dec_info[1][5],
                    "rainfall" : dec_info[1][6],
                    "windspeed" : dec_info[1][9]
                }
            }
        },
        {
            "destination" : "Sandakan",
            "months" : {
                "january" :{
                    "probability" : jan_prob_list[2],
                    "avg_temp" : jan_info[2][1],
                    "clearer" : jan_info[2][4],
                    "precipitation" : jan_info[2][5],
                    "rainfall" : jan_info[2][6],
                    "windspeed" : jan_info[2][9]
                },
                "february" :{
                    "probability" : jan_prob_list[2],
                    "avg_temp" : feb_info[2][1],
                    "clearer" : feb_info[2][4],
                    "precipitation" : feb_info[2][5],
                    "rainfall" : feb_info[2][6],
                    "windspeed" : feb_info[2][9]
                },
                "march" :{
                    "probability" : mar_prob_list[2],
                    "avg_temp" : mar_info[2][1],
                    "clearer" : mar_info[2][4],
                    "precipitation" : mar_info[2][5],
                    "rainfall" : mar_info[2][6],
                    "windspeed" : mar_info[2][9]
                },
                "april" :{
                    "probability" : apr_prob_list[2],
                    "avg_temp" : apr_info[2][1],
                    "clearer" : apr_info[2][4],
                    "precipitation" : apr_info[2][5],
                    "rainfall" : apr_info[2][6],
                    "windspeed" : apr_info[2][9]
                },
                "may" :{
                    "probability" : may_prob_list[2],
                    "avg_temp" : may_info[2][1],
                    "clearer" : may_info[2][4],
                    "precipitation" : may_info[2][5],
                    "rainfall" : may_info[2][6],
                    "windspeed" : may_info[2][9]
                },
                "june" :{
                    "probability" : jun_prob_list[2],
                    "avg_temp" : jun_info[2][1],
                    "clearer" : jun_info[2][4],
                    "precipitation" : jun_info[2][5],
                    "rainfall" : jun_info[2][6],
                    "windspeed" : jun_info[2][9]
                },
                "july" :{
                    "probability" : jul_prob_list[2],
                    "avg_temp" : jul_info[2][1],
                    "clearer" : jul_info[2][4],
                    "precipitation" : jul_info[2][5],
                    "rainfall" : jul_info[2][6],
                    "windspeed" : jul_info[2][9]
                },
                "august" :{
                    "probability" : aug_prob_list[2],
                    "avg_temp" : aug_info[2][1],
                    "clearer" : aug_info[2][4],
                    "precipitation" : aug_info[2][5],
                    "rainfall" : aug_info[2][6],
                    "windspeed" : aug_info[2][9]
                },
                "september" :{
                    "probability" : sep_prob_list[2],
                    "avg_temp" : sep_info[2][1],
                    "clearer" : sep_info[2][4],
                    "precipitation" : sep_info[2][5],
                    "rainfall" : sep_info[2][6],
                    "windspeed" : sep_info[2][9]
                },
                "october" :{
                    "probability" : oct_prob_list[2],
                    "avg_temp" : oct_info[2][1],
                    "clearer" : oct_info[2][4],
                    "precipitation" : oct_info[2][5],
                    "rainfall" : oct_info[2][6],
                    "windspeed" : oct_info[2][9]
                },
                "november" :{
                    "probability" : nov_prob_list[2],
                    "avg_temp" : nov_info[2][1],
                    "clearer" : nov_info[2][4],
                    "precipitation" : nov_info[2][5],
                    "rainfall" : nov_info[2][6],
                    "windspeed" : nov_info[2][9]
                },
                "december" :{
                    "probability" : dec_prob_list[2],
                    "avg_temp" : dec_info[2][1],
                    "clearer" : dec_info[2][4],
                    "precipitation" : dec_info[2][5],
                    "rainfall" : dec_info[2][6],
                    "windspeed" : dec_info[2][9]
                }
            }
        },
        {
            "destination" : "Kuching",
            "months" : {
                "january" :{
                    "probability" : jan_prob_list[3],
                    "avg_temp" : jan_info[3][1],
                    "clearer" : jan_info[3][4],
                    "precipitation" : jan_info[3][5],
                    "rainfall" : jan_info[3][6],
                    "windspeed" : jan_info[3][9]
                },
                "february" :{
                    "probability" : jan_prob_list[3],
                    "avg_temp" : feb_info[3][1],
                    "clearer" : feb_info[3][4],
                    "precipitation" : feb_info[3][5],
                    "rainfall" : feb_info[3][6],
                    "windspeed" : feb_info[3][9]
                },
                "march" :{
                    "probability" : mar_prob_list[3],
                    "avg_temp" : mar_info[3][1],
                    "clearer" : mar_info[3][4],
                    "precipitation" : mar_info[3][5],
                    "rainfall" : mar_info[3][6],
                    "windspeed" : mar_info[3][9]
                },
                "april" :{
                    "probability" : apr_prob_list[3],
                    "avg_temp" : apr_info[3][1],
                    "clearer" : apr_info[3][4],
                    "precipitation" : apr_info[3][5],
                    "rainfall" : apr_info[3][6],
                    "windspeed" : apr_info[3][9]
                },
                "may" :{
                    "probability" : may_prob_list[3],
                    "avg_temp" : may_info[3][1],
                    "clearer" : may_info[3][4],
                    "precipitation" : may_info[3][5],
                    "rainfall" : may_info[3][6],
                    "windspeed" : may_info[3][9]
                },
                "june" :{
                    "probability" : jun_prob_list[3],
                    "avg_temp" : jun_info[3][1],
                    "clearer" : jun_info[3][4],
                    "precipitation" : jun_info[3][5],
                    "rainfall" : jun_info[3][6],
                    "windspeed" : jun_info[3][9]
                },
                "july" :{
                    "probability" : jul_prob_list[3],
                    "avg_temp" : jul_info[3][1],
                    "clearer" : jul_info[3][4],
                    "precipitation" : jul_info[3][5],
                    "rainfall" : jul_info[3][6],
                    "windspeed" : jul_info[3][9]
                },
                "august" :{
                    "probability" : aug_prob_list[3],
                    "avg_temp" : aug_info[3][1],
                    "clearer" : aug_info[3][4],
                    "precipitation" : aug_info[3][5],
                    "rainfall" : aug_info[3][6],
                    "windspeed" : aug_info[3][9]
                },
                "september" :{
                    "probability" : sep_prob_list[3],
                    "avg_temp" : sep_info[3][1],
                    "clearer" : sep_info[3][4],
                    "precipitation" : sep_info[3][5],
                    "rainfall" : sep_info[3][6],
                    "windspeed" : sep_info[3][9]
                },
                "october" :{
                    "probability" : oct_prob_list[3],
                    "avg_temp" : oct_info[3][1],
                    "clearer" : oct_info[3][4],
                    "precipitation" : oct_info[3][5],
                    "rainfall" : oct_info[3][6],
                    "windspeed" : oct_info[3][9]
                },
                "november" :{
                    "probability" : nov_prob_list[3],
                    "avg_temp" : nov_info[3][1],
                    "clearer" : nov_info[3][4],
                    "precipitation" : nov_info[3][5],
                    "rainfall" : nov_info[3][6],
                    "windspeed" : nov_info[3][9]
                },
                "december" :{
                    "probability" : dec_prob_list[3],
                    "avg_temp" : dec_info[3][1],
                    "clearer" : dec_info[3][4],
                    "precipitation" : dec_info[3][5],
                    "rainfall" : dec_info[3][6],
                    "windspeed" : dec_info[3][9]
                }
            }
        },
        {
            "destination" : "Miri",
            "months" : {
                "january" :{
                    "probability" : jan_prob_list[4],
                    "avg_temp" : jan_info[4][1],
                    "clearer" : jan_info[4][4],
                    "precipitation" : jan_info[4][5],
                    "rainfall" : jan_info[4][6],
                    "windspeed" : jan_info[4][9]
                },
                "february" :{
                    "probability" : jan_prob_list[4],
                    "avg_temp" : feb_info[4][1],
                    "clearer" : feb_info[4][4],
                    "precipitation" : feb_info[4][5],
                    "rainfall" : feb_info[4][6],
                    "windspeed" : feb_info[4][9]
                },
                "march" :{
                    "probability" : mar_prob_list[4],
                    "avg_temp" : mar_info[4][1],
                    "clearer" : mar_info[4][4],
                    "precipitation" : mar_info[4][5],
                    "rainfall" : mar_info[4][6],
                    "windspeed" : mar_info[4][9]
                },
                "april" :{
                    "probability" : apr_prob_list[4],
                    "avg_temp" : apr_info[4][1],
                    "clearer" : apr_info[4][4],
                    "precipitation" : apr_info[4][5],
                    "rainfall" : apr_info[4][6],
                    "windspeed" : apr_info[4][9]
                },
                "may" :{
                    "probability" : may_prob_list[4],
                    "avg_temp" : may_info[4][1],
                    "clearer" : may_info[4][4],
                    "precipitation" : may_info[4][5],
                    "rainfall" : may_info[4][6],
                    "windspeed" : may_info[4][9]
                },
                "june" :{
                    "probability" : jun_prob_list[4],
                    "avg_temp" : jun_info[4][1],
                    "clearer" : jun_info[4][4],
                    "precipitation" : jun_info[4][5],
                    "rainfall" : jun_info[4][6],
                    "windspeed" : jun_info[4][9]
                },
                "july" :{
                    "probability" : jul_prob_list[4],
                    "avg_temp" : jul_info[4][1],
                    "clearer" : jul_info[4][4],
                    "precipitation" : jul_info[4][5],
                    "rainfall" : jul_info[4][6],
                    "windspeed" : jul_info[4][9]
                },
                "august" :{
                    "probability" : aug_prob_list[4],
                    "avg_temp" : aug_info[4][1],
                    "clearer" : aug_info[4][4],
                    "precipitation" : aug_info[4][5],
                    "rainfall" : aug_info[4][6],
                    "windspeed" : aug_info[4][9]
                },
                "september" :{
                    "probability" : sep_prob_list[4],
                    "avg_temp" : sep_info[4][1],
                    "clearer" : sep_info[4][4],
                    "precipitation" : sep_info[4][5],
                    "rainfall" : sep_info[4][6],
                    "windspeed" : sep_info[4][9]
                },
                "october" :{
                    "probability" : oct_prob_list[4],
                    "avg_temp" : oct_info[4][1],
                    "clearer" : oct_info[4][4],
                    "precipitation" : oct_info[4][5],
                    "rainfall" : oct_info[4][6],
                    "windspeed" : oct_info[4][9]
                },
                "november" :{
                    "probability" : nov_prob_list[4],
                    "avg_temp" : nov_info[4][1],
                    "clearer" : nov_info[4][4],
                    "precipitation" : nov_info[4][5],
                    "rainfall" : nov_info[4][6],
                    "windspeed" : nov_info[4][9]
                },
                "december" :{
                    "probability" : dec_prob_list[4],
                    "avg_temp" : dec_info[4][1],
                    "clearer" : dec_info[4][4],
                    "precipitation" : dec_info[4][5],
                    "rainfall" : dec_info[4][6],
                    "windspeed" : dec_info[4][9]
                }
            }
        },
        {
            "destination" : "Sibu",
            "months" :{
                "january" :{
                    "probability" : jan_prob_list[5],
                    "avg_temp" : jan_info[5][1],
                    "clearer" : jan_info[5][4],
                    "precipitation" : jan_info[5][5],
                    "rainfall" : jan_info[5][6],
                    "windspeed" : jan_info[5][9]
                },
                "february" :{
                    "probability" : jan_prob_list[5],
                    "avg_temp" : feb_info[5][1],
                    "clearer" : feb_info[5][4],
                    "precipitation" : feb_info[5][5],
                    "rainfall" : feb_info[5][6],
                    "windspeed" : feb_info[5][9]
                },
                "march" :{
                    "probability" : mar_prob_list[5],
                    "avg_temp" : mar_info[5][1],
                    "clearer" : mar_info[5][4],
                    "precipitation" : mar_info[5][5],
                    "rainfall" : mar_info[5][6],
                    "windspeed" : mar_info[5][9]
                },
                "april" :{
                    "probability" : apr_prob_list[5],
                    "avg_temp" : apr_info[5][1],
                    "clearer" : apr_info[5][4],
                    "precipitation" : apr_info[5][5],
                    "rainfall" : apr_info[5][6],
                    "windspeed" : apr_info[5][9]
                },
                "may" :{
                    "probability" : may_prob_list[5],
                    "avg_temp" : may_info[5][1],
                    "clearer" : may_info[5][4],
                    "precipitation" : may_info[5][5],
                    "rainfall" : may_info[5][6],
                    "windspeed" : may_info[5][9]
                },
                "june" :{
                    "probability" : jun_prob_list[5],
                    "avg_temp" : jun_info[5][1],
                    "clearer" : jun_info[5][4],
                    "precipitation" : jun_info[5][5],
                    "rainfall" : jun_info[5][6],
                    "windspeed" : jun_info[5][9]
                },
                "july" :{
                    "probability" : jul_prob_list[5],
                    "avg_temp" : jul_info[5][1],
                    "clearer" : jul_info[5][4],
                    "precipitation" : jul_info[5][5],
                    "rainfall" : jul_info[5][6],
                    "windspeed" : jul_info[5][9]
                },
                "august" :{
                    "probability" : aug_prob_list[5],
                    "avg_temp" : aug_info[5][1],
                    "clearer" : aug_info[5][4],
                    "precipitation" : aug_info[5][5],
                    "rainfall" : aug_info[5][6],
                    "windspeed" : aug_info[5][9]
                },
                "september" :{
                    "probability" : sep_prob_list[5],
                    "avg_temp" : sep_info[5][1],
                    "clearer" : sep_info[5][4],
                    "precipitation" : sep_info[5][5],
                    "rainfall" : sep_info[5][6],
                    "windspeed" : sep_info[5][9]
                },
                "october" :{
                    "probability" : oct_prob_list[5],
                    "avg_temp" : oct_info[5][1],
                    "clearer" : oct_info[5][4],
                    "precipitation" : oct_info[5][5],
                    "rainfall" : oct_info[5][6],
                    "windspeed" : oct_info[5][9]
                },
                "november" :{
                    "probability" : nov_prob_list[5],
                    "avg_temp" : nov_info[5][1],
                    "clearer" : nov_info[5][4],
                    "precipitation" : nov_info[5][5],
                    "rainfall" : nov_info[5][6],
                    "windspeed" : nov_info[5][9]
                },
                "december" :{
                    "probability" : dec_prob_list[5],
                    "avg_temp" : dec_info[5][1],
                    "clearer" : dec_info[5][4],
                    "precipitation" : dec_info[5][5],
                    "rainfall" : dec_info[5][6],
                    "windspeed" : dec_info[5][9]
                }
            }
        },
        {
            "destination" : "Kota Bharu",
            "months" : {
                "january" :{
                    "probability" : jan_prob_list[6],
                    "avg_temp" : jan_info[6][1],
                    "clearer" : jan_info[6][4],
                    "precipitation" : jan_info[6][5],
                    "rainfall" : jan_info[6][6],
                    "windspeed" : jan_info[6][9]
                },
                "february" :{
                    "probability" : jan_prob_list[6],
                    "avg_temp" : feb_info[6][1],
                    "clearer" : feb_info[6][4],
                    "precipitation" : feb_info[6][5],
                    "rainfall" : feb_info[6][6],
                    "windspeed" : feb_info[6][9]
                },
                "march" :{
                    "probability" : mar_prob_list[6],
                    "avg_temp" : mar_info[6][1],
                    "clearer" : mar_info[6][4],
                    "precipitation" : mar_info[6][5],
                    "rainfall" : mar_info[6][6],
                    "windspeed" : mar_info[6][9]
                },
                "april" :{
                    "probability" : apr_prob_list[6],
                    "avg_temp" : apr_info[6][1],
                    "clearer" : apr_info[6][4],
                    "precipitation" : apr_info[6][5],
                    "rainfall" : apr_info[6][6],
                    "windspeed" : apr_info[6][9]
                },
                "may" :{
                    "probability" : may_prob_list[6],
                    "avg_temp" : may_info[6][1],
                    "clearer" : may_info[6][4],
                    "precipitation" : may_info[6][5],
                    "rainfall" : may_info[6][6],
                    "windspeed" : may_info[6][9]
                },
                "june" :{
                    "probability" : jun_prob_list[6],
                    "avg_temp" : jun_info[6][1],
                    "clearer" : jun_info[6][4],
                    "precipitation" : jun_info[6][5],
                    "rainfall" : jun_info[6][6],
                    "windspeed" : jun_info[6][9]
                },
                "july" :{
                    "probability" : jul_prob_list[6],
                    "avg_temp" : jul_info[6][1],
                    "clearer" : jul_info[6][4],
                    "precipitation" : jul_info[6][5],
                    "rainfall" : jul_info[6][6],
                    "windspeed" : jul_info[6][9]
                },
                "august" :{
                    "probability" : aug_prob_list[6],
                    "avg_temp" : aug_info[6][1],
                    "clearer" : aug_info[6][4],
                    "precipitation" : aug_info[6][5],
                    "rainfall" : aug_info[6][6],
                    "windspeed" : aug_info[6][9]
                },
                "september" :{
                    "probability" : sep_prob_list[6],
                    "avg_temp" : sep_info[6][1],
                    "clearer" : sep_info[6][4],
                    "precipitation" : sep_info[6][5],
                    "rainfall" : sep_info[6][6],
                    "windspeed" : sep_info[6][9]
                },
                "october" :{
                    "probability" : oct_prob_list[6],
                    "avg_temp" : oct_info[6][1],
                    "clearer" : oct_info[6][4],
                    "precipitation" : oct_info[6][5],
                    "rainfall" : oct_info[6][6],
                    "windspeed" : oct_info[6][9]
                },
                "november" :{
                    "probability" : nov_prob_list[6],
                    "avg_temp" : nov_info[6][1],
                    "clearer" : nov_info[6][4],
                    "precipitation" : nov_info[6][5],
                    "rainfall" : nov_info[6][6],
                    "windspeed" : nov_info[6][9]
                },
                "december" :{
                    "probability" : dec_prob_list[6],
                    "avg_temp" : dec_info[6][1],
                    "clearer" : dec_info[6][4],
                    "precipitation" : dec_info[6][5],
                    "rainfall" : dec_info[6][6],
                    "windspeed" : dec_info[6][9]
                }
            }
        },
        {
            "destination" : "Kuantan",
            "months" : {
                "january" :{
                    "probability" : jan_prob_list[7],
                    "avg_temp" : jan_info[7][1],
                    "clearer" : jan_info[7][4],
                    "precipitation" : jan_info[7][5],
                    "rainfall" : jan_info[7][6],
                    "windspeed" : jan_info[7][9]
                },
                "february" :{
                    "probability" : jan_prob_list[7],
                    "avg_temp" : feb_info[7][1],
                    "clearer" : feb_info[7][4],
                    "precipitation" : feb_info[7][5],
                    "rainfall" : feb_info[7][6],
                    "windspeed" : feb_info[7][9],
                },
                "march" :{
                    "probability" : mar_prob_list[7],
                    "avg_temp" : mar_info[7][1],
                    "clearer" : mar_info[7][4],
                    "precipitation" : mar_info[7][5],
                    "rainfall" : mar_info[7][6],
                    "windspeed" : mar_info[7][9]
                },
                "april" :{
                    "probability" : apr_prob_list[7],
                    "avg_temp" : apr_info[7][1],
                    "clearer" : apr_info[7][4],
                    "precipitation" : apr_info[7][5],
                    "rainfall" : apr_info[7][6],
                    "windspeed" : apr_info[7][9]
                },
                "may" :{
                    "probability" : may_prob_list[7],
                    "avg_temp" : may_info[7][1],
                    "clearer" : may_info[7][4],
                    "precipitation" : may_info[7][5],
                    "rainfall" : may_info[7][6],
                    "windspeed" : may_info[7][9]
                },
                "june" :{
                    "probability" : jun_prob_list[7],
                    "avg_temp" : jun_info[7][1],
                    "clearer" : jun_info[7][4],
                    "precipitation" : jun_info[7][5],
                    "rainfall" : jun_info[7][6],
                    "windspeed" : jun_info[7][9]
                },
                "july" :{
                    "probability" : jul_prob_list[7],
                    "avg_temp" : jul_info[7][1],
                    "clearer" : jul_info[7][4],
                    "precipitation" : jul_info[7][5],
                    "rainfall" : jul_info[7][6],
                    "windspeed" : jul_info[7][9]
                },
                "august" :{
                    "probability" : aug_prob_list[7],
                    "avg_temp" : aug_info[7][1],
                    "clearer" : aug_info[7][4],
                    "precipitation" : aug_info[7][5],
                    "rainfall" : aug_info[7][6],
                    "windspeed" : aug_info[7][9]
                },
                "september" :{
                    "probability" : sep_prob_list[7],
                    "avg_temp" : sep_info[7][1],
                    "clearer" : sep_info[7][4],
                    "precipitation" : sep_info[7][5],
                    "rainfall" : sep_info[7][6],
                    "windspeed" : sep_info[7][9]
                },
                "october" :{
                    "probability" : oct_prob_list[7],
                    "avg_temp" : oct_info[7][1],
                    "clearer" : oct_info[7][4],
                    "precipitation" : oct_info[7][5],
                    "rainfall" : oct_info[7][6],
                    "windspeed" : oct_info[7][9]
                },
                "november" :{
                    "probability" : nov_prob_list[7],
                    "avg_temp" : nov_info[7][1],
                    "clearer" : nov_info[7][4],
                    "precipitation" : nov_info[7][5],
                    "rainfall" : nov_info[7][6],
                    "windspeed" : nov_info[7][9]
                },
                "december" :{
                    "probability" : dec_prob_list[7],
                    "avg_temp" : dec_info[7][1],
                    "clearer" : dec_info[7][4],
                    "precipitation" : dec_info[7][5],
                    "rainfall" : dec_info[7][6],
                    "windspeed" : dec_info[7][9]
                }
            }
        },
        {
            "destination" : "Kuala Terengganu",
            "months" : {
                "january" :{
                    "probability" : jan_prob_list[8],
                    "avg_temp" : jan_info[8][1],
                    "clearer" : jan_info[8][4],
                    "precipitation" : jan_info[8][5],
                    "rainfall" : jan_info[8][6],
                    "windspeed" : jan_info[8][9]
                },
                "february" :{
                    "probability" : jan_prob_list[8],
                    "avg_temp" : feb_info[8][1],
                    "clearer" : feb_info[8][4],
                    "precipitation" : feb_info[8][5],
                    "rainfall" : feb_info[8][6],
                    "windspeed" : feb_info[8][9]
                },
                "march" :{
                    "probability" : mar_prob_list[8],
                    "avg_temp" : mar_info[8][1],
                    "clearer" : mar_info[8][4],
                    "precipitation" : mar_info[8][5],
                    "rainfall" : mar_info[8][6],
                    "windspeed" : mar_info[8][9]
                },
                "april" :{
                    "probability" : apr_prob_list[8],
                    "avg_temp" : apr_info[8][1],
                    "clearer" : apr_info[8][4],
                    "precipitation" : apr_info[8][5],
                    "rainfall" : apr_info[8][6],
                    "windspeed" : apr_info[8][9]
                },
                "may" :{
                    "probability" : may_prob_list[8],
                    "avg_temp" : may_info[8][1],
                    "clearer" : may_info[8][4],
                    "precipitation" : may_info[8][5],
                    "rainfall" : may_info[8][6],
                    "windspeed" : may_info[8][9]
                },
                "june" :{
                    "probability" : jun_prob_list[8],
                    "avg_temp" : jun_info[8][1],
                    "clearer" : jun_info[8][4],
                    "precipitation" : jun_info[8][5],
                    "rainfall" : jun_info[8][6],
                    "windspeed" : jun_info[8][9]
                },
                "july" :{
                    "probability" : jul_prob_list[8],
                    "avg_temp" : jul_info[8][1],
                    "clearer" : jul_info[8][4],
                    "precipitation" : jul_info[8][5],
                    "rainfall" : jul_info[8][6],
                    "windspeed" : jul_info[8][9]
                },
                "august" :{
                    "probability" : aug_prob_list[8],
                    "avg_temp" : aug_info[8][1],
                    "clearer" : aug_info[8][4],
                    "precipitation" : aug_info[8][5],
                    "rainfall" : aug_info[8][6],
                    "windspeed" : aug_info[8][9]
                },
                "september" :{
                    "probability" : sep_prob_list[8],
                    "avg_temp" : sep_info[8][1],
                    "clearer" : sep_info[8][4],
                    "precipitation" : sep_info[8][5],
                    "rainfall" : sep_info[8][6],
                    "windspeed" : sep_info[8][9]
                },
                "october" :{
                    "probability" : oct_prob_list[8],
                    "avg_temp" : oct_info[8][1],
                    "clearer" : oct_info[8][4],
                    "precipitation" : oct_info[8][5],
                    "rainfall" : oct_info[8][6],
                    "windspeed" : oct_info[8][9]
                },
                "november" :{
                    "probability" : nov_prob_list[8],
                    "avg_temp" : nov_info[8][1],
                    "clearer" : nov_info[8][4],
                    "precipitation" : nov_info[8][5],
                    "rainfall" : nov_info[8][6],
                    "windspeed" : nov_info[8][9]
                },
                "december" :{
                    "probability" : dec_prob_list[8],
                    "avg_temp" : dec_info[8][1],
                    "clearer" : dec_info[8][4],
                    "precipitation" : dec_info[8][5],
                    "rainfall" : dec_info[8][6],
                    "windspeed" : dec_info[8][9]
                }
            }
        },
        {
            "destination" : "Kangar",
            "months" : {
                "january" :{
                    "probability" : jan_prob_list[9],
                    "avg_temp" : jan_info[9][1],
                    "clearer" : jan_info[9][4],
                    "precipitation" : jan_info[9][5],
                    "rainfall" : jan_info[9][6],
                    "windspeed" : jan_info[9][9]
                },
                "february" :{
                    "probability" : jan_prob_list[9],
                    "avg_temp" : feb_info[9][1],
                    "clearer" : feb_info[9][4],
                    "precipitation" : feb_info[9][5],
                    "rainfall" : feb_info[9][6],
                    "windspeed" : feb_info[9][9]
                },
                "march" :{
                    "probability" : mar_prob_list[9],
                    "avg_temp" : mar_info[9][1],
                    "clearer" : mar_info[9][4],
                    "precipitation" : mar_info[9][5],
                    "rainfall" : mar_info[9][6],
                    "windspeed" : mar_info[9][9]
                },
                "april" :{
                    "probability" : apr_prob_list[9],
                    "avg_temp" : apr_info[9][1],
                    "clearer" : apr_info[9][4],
                    "precipitation" : apr_info[9][5],
                    "rainfall" : apr_info[9][6],
                    "windspeed" : apr_info[9][9]
                },
                "may" :{
                    "probability" : may_prob_list[9],
                    "avg_temp" : may_info[9][1],
                    "clearer" : may_info[9][4],
                    "precipitation" : may_info[9][5],
                    "rainfall" : may_info[9][6],
                    "windspeed" : may_info[9][9]
                },
                "june" :{
                    "probability" : jun_prob_list[9],
                    "avg_temp" : jun_info[9][1],
                    "clearer" : jun_info[9][4],
                    "precipitation" : jun_info[9][5],
                    "rainfall" : jun_info[9][6],
                    "windspeed" : jun_info[9][9]
                },
                "july" :{
                    "probability" : jul_prob_list[9],
                    "avg_temp" : jul_info[9][1],
                    "clearer" : jul_info[9][4],
                    "precipitation" : jul_info[9][5],
                    "rainfall" : jul_info[9][6],
                    "windspeed" : jul_info[9][9]
                },
                "august" :{
                    "probability" : aug_prob_list[9],
                    "avg_temp" : aug_info[9][1],
                    "clearer" : aug_info[9][4],
                    "precipitation" : aug_info[9][5],
                    "rainfall" : aug_info[9][6],
                    "windspeed" : aug_info[9][9]
                },
                "september" :{
                    "probability" : sep_prob_list[9],
                    "avg_temp" : sep_info[9][1],
                    "clearer" : sep_info[9][4],
                    "precipitation" : sep_info[9][5],
                    "rainfall" : sep_info[9][6],
                    "windspeed" : sep_info[9][9]
                },
                "october" :{
                    "probability" : oct_prob_list[9],
                    "avg_temp" : oct_info[9][1],
                    "clearer" : oct_info[9][4],
                    "precipitation" : oct_info[9][5],
                    "rainfall" : oct_info[9][6],
                    "windspeed" : oct_info[9][9]
                },
                "november" :{
                    "probability" : nov_prob_list[9],
                    "avg_temp" : nov_info[9][1],
                    "clearer" : nov_info[9][4],
                    "precipitation" : nov_info[9][5],
                    "rainfall" : nov_info[9][6],
                    "windspeed" : nov_info[9][9]
                },
                "december" :{
                    "probability" : dec_prob_list[9],
                    "avg_temp" : dec_info[9][1],
                    "clearer" : dec_info[9][4],
                    "precipitation" : dec_info[9][5],
                    "rainfall" : dec_info[9][6],
                    "windspeed" : dec_info[9][9]
                }
            }
        },
        {
            "destination" : "Alor Setar",
            "months" : {
                "january" :{
                    "probability" : jan_prob_list[10],
                    "avg_temp" : jan_info[10][1],
                    "clearer" : jan_info[10][4],
                    "precipitation" : jan_info[10][5],
                    "rainfall" : jan_info[10][6],
                    "windspeed" : jan_info[10][9]
                },
                "february" :{
                    "probability" : jan_prob_list[10],
                    "avg_temp" : feb_info[10][1],
                    "clearer" : feb_info[10][4],
                    "precipitation" : feb_info[10][5],
                    "rainfall" : feb_info[10][6],
                    "windspeed" : feb_info[10][9]
                },
                "march" :{
                    "probability" : mar_prob_list[10],
                    "avg_temp" : mar_info[10][1],
                    "clearer" : mar_info[10][4],
                    "precipitation" : mar_info[10][5],
                    "rainfall" : mar_info[10][6],
                    "windspeed" : mar_info[10][9]
                },
                "april" :{
                    "probability" : apr_prob_list[10],
                    "avg_temp" : apr_info[10][1],
                    "clearer" : apr_info[10][4],
                    "precipitation" : apr_info[10][5],
                    "rainfall" : apr_info[10][6],
                    "windspeed" : apr_info[10][9]
                },
                "may" :{
                    "probability" : may_prob_list[10],
                    "avg_temp" : may_info[10][1],
                    "clearer" : may_info[10][4],
                    "precipitation" : may_info[10][5],
                    "rainfall" : may_info[10][6],
                    "windspeed" : may_info[10][9]
                },
                "june" :{
                    "probability" : jun_prob_list[10],
                    "avg_temp" : jun_info[10][1],
                    "clearer" : jun_info[10][4],
                    "precipitation" : jun_info[10][5],
                    "rainfall" : jun_info[10][6],
                    "windspeed" : jun_info[10][9]
                },
                "july" :{
                    "probability" : jul_prob_list[10],
                    "avg_temp" : jul_info[10][1],
                    "clearer" : jul_info[10][4],
                    "precipitation" : jul_info[10][5],
                    "rainfall" : jul_info[10][6],
                    "windspeed" : jul_info[10][9]
                },
                "august" :{
                    "probability" : aug_prob_list[10],
                    "avg_temp" : aug_info[10][1],
                    "clearer" : aug_info[10][4],
                    "precipitation" : aug_info[10][5],
                    "rainfall" : aug_info[10][6],
                    "windspeed" : aug_info[10][9]
                },
                "september" :{
                    "probability" : sep_prob_list[10],
                    "avg_temp" : sep_info[10][1],
                    "clearer" : sep_info[10][4],
                    "precipitation" : sep_info[10][5],
                    "rainfall" : sep_info[10][6],
                    "windspeed" : sep_info[10][9]
                },
                "october" :{
                    "probability" : oct_prob_list[10],
                    "avg_temp" : oct_info[10][1],
                    "clearer" : oct_info[10][4],
                    "precipitation" : oct_info[10][5],
                    "rainfall" : oct_info[10][6],
                    "windspeed" : oct_info[10][9]
                },
                "november" :{
                    "probability" : nov_prob_list[10],
                    "avg_temp" : nov_info[10][1],
                    "clearer" : nov_info[10][4],
                    "precipitation" : nov_info[10][5],
                    "rainfall" : nov_info[10][6],
                    "windspeed" : nov_info[10][9]
                },
                "december" :{
                    "probability" : dec_prob_list[10],
                    "avg_temp" : dec_info[10][1],
                    "clearer" : dec_info[10][4],
                    "precipitation" : dec_info[10][5],
                    "rainfall" : dec_info[10][6],
                    "windspeed" : dec_info[10][9]
                }
            }
        },
        {
            "destination" : "Georgetown",
            "months" : {
                "january" :{
                    "probability" : jan_prob_list[11],
                    "avg_temp" : jan_info[11][1],
                    "clearer" : jan_info[11][4],
                    "precipitation" : jan_info[11][5],
                    "rainfall" : jan_info[11][6],
                    "windspeed" : jan_info[11][9]
                },
                "february" :{
                    "probability" : jan_prob_list[11],
                    "avg_temp" : feb_info[11][1],
                    "clearer" : feb_info[11][4],
                    "precipitation" : feb_info[11][5],
                    "rainfall" : feb_info[11][6],
                    "windspeed" : feb_info[11][9]
                },
                "march" :{
                    "probability" : mar_prob_list[11],
                    "avg_temp" : mar_info[11][1],
                    "clearer" : mar_info[11][4],
                    "precipitation" : mar_info[11][5],
                    "rainfall" : mar_info[11][6],
                    "windspeed" : mar_info[11][9]
                },
                "april" :{
                    "probability" : apr_prob_list[11],
                    "avg_temp" : apr_info[11][1],
                    "clearer" : apr_info[11][4],
                    "precipitation" : apr_info[11][5],
                    "rainfall" : apr_info[11][6],
                    "windspeed" : apr_info[11][9]
                },
                "may" :{
                    "probability" : may_prob_list[11],
                    "avg_temp" : may_info[11][1],
                    "clearer" : may_info[11][4],
                    "precipitation" : may_info[11][5],
                    "rainfall" : may_info[11][6],
                    "windspeed" : may_info[11][9]
                },
                "june" :{
                    "probability" : jun_prob_list[11],
                    "avg_temp" : jun_info[11][1],
                    "clearer" : jun_info[11][4],
                    "precipitation" : jun_info[11][5],
                    "rainfall" : jun_info[11][6],
                    "windspeed" : jun_info[11][9]
                },
                "july" :{
                    "probability" : jul_prob_list[11],
                    "avg_temp" : jul_info[11][1],
                    "clearer" : jul_info[11][4],
                    "precipitation" : jul_info[11][5],
                    "rainfall" : jul_info[11][6],
                    "windspeed" : jul_info[11][9]
                },
                "august" :{
                    "probability" : aug_prob_list[11],
                    "avg_temp" : aug_info[11][1],
                    "clearer" : aug_info[11][4],
                    "precipitation" : aug_info[11][5],
                    "rainfall" : aug_info[11][6],
                    "windspeed" : aug_info[11][9]
                },
                "september" :{
                    "probability" : sep_prob_list[11],
                    "avg_temp" : sep_info[11][1],
                    "clearer" : sep_info[11][4],
                    "precipitation" : sep_info[11][5],
                    "rainfall" : sep_info[11][6],
                    "windspeed" : sep_info[11][9]
                },
                "october" :{
                    "probability" : oct_prob_list[11],
                    "avg_temp" : oct_info[11][1],
                    "clearer" : oct_info[11][4],
                    "precipitation" : oct_info[11][5],
                    "rainfall" : oct_info[11][6],
                    "windspeed" : oct_info[11][9]
                },
                "november" :{
                    "probability" : nov_prob_list[11],
                    "avg_temp" : nov_info[11][1],
                    "clearer" : nov_info[11][4],
                    "precipitation" : nov_info[11][5],
                    "rainfall" : nov_info[11][6],
                    "windspeed" : nov_info[11][9]
                },
                "december" :{
                    "probability" : dec_prob_list[11],
                    "avg_temp" : dec_info[11][1],
                    "clearer" : dec_info[11][4],
                    "precipitation" : dec_info[11][5],
                    "rainfall" : dec_info[11][6],
                    "windspeed" : dec_info[11][9]
                }
            }
        },
        {
            "destination" : "Ipoh",
            "months" : {
                "january" : {
                    "probability" : jan_prob_list[12],
                    "avg_temp" : jan_info[12][1],
                    "clearer" : jan_info[12][4],
                    "precipitation" : jan_info[12][5],
                    "rainfall" : jan_info[12][6],
                    "windspeed" : jan_info[12][9]
                },
                "february" :{
                    "probability" : jan_prob_list[12],
                    "avg_temp" : feb_info[12][1],
                    "clearer" : feb_info[12][4],
                    "precipitation" : feb_info[12][5],
                    "rainfall" : feb_info[12][6],
                    "windspeed" : feb_info[12][9]
                },
                "march" :{
                    "probability" : mar_prob_list[12],
                    "avg_temp" : mar_info[12][1],
                    "clearer" : mar_info[12][4],
                    "precipitation" : mar_info[12][5],
                    "rainfall" : mar_info[12][6],
                    "windspeed" : mar_info[12][9]
                },
                "april" :{
                    "probability" : apr_prob_list[12],
                    "avg_temp" : apr_info[12][1],
                    "clearer" : apr_info[12][4],
                    "precipitation" : apr_info[12][5],
                    "rainfall" : apr_info[12][6],
                    "windspeed" : apr_info[12][9]
                },
                "may" :{
                    "probability" : may_prob_list[12],
                    "avg_temp" : may_info[12][1],
                    "clearer" : may_info[12][4],
                    "precipitation" : may_info[12][5],
                    "rainfall" : may_info[12][6],
                    "windspeed" : may_info[12][9],
                },
                "june" :{
                    "probability" : jun_prob_list[12],
                    "avg_temp" : jun_info[12][1],
                    "clearer" : jun_info[12][4],
                    "precipitation" : jun_info[12][5],
                    "rainfall" : jun_info[12][6],
                    "windspeed" : jun_info[12][9]
                },
                "july" :{
                    "probability" : jul_prob_list[12],
                    "avg_temp" : jul_info[12][1],
                    "clearer" : jul_info[12][4],
                    "precipitation" : jul_info[12][5],
                    "rainfall" : jul_info[12][6],
                    "windspeed" : jul_info[12][9]
                },
                "august" :{
                    "probability" : aug_prob_list[12],
                    "avg_temp" : aug_info[12][1],
                    "clearer" : aug_info[12][4],
                    "precipitation" : aug_info[12][5],
                    "rainfall" : aug_info[12][6],
                    "windspeed" : aug_info[12][9]
                },
                "september" :{
                    "probability" : sep_prob_list[12],
                    "avg_temp" : sep_info[12][1],
                    "clearer" : sep_info[12][4],
                    "precipitation" : sep_info[12][5],
                    "rainfall" : sep_info[12][6],
                    "windspeed" : sep_info[12][9]
                },
                "october" :{
                    "probability" : oct_prob_list[12],
                    "avg_temp" : oct_info[12][1],
                    "clearer" : oct_info[12][4],
                    "precipitation" : oct_info[12][5],
                    "rainfall" : oct_info[12][6],
                    "windspeed" : oct_info[12][9]
                },
                "november" :{
                    "probability" : nov_prob_list[12],
                    "avg_temp" : nov_info[12][1],
                    "clearer" : nov_info[12][4],
                    "precipitation" : nov_info[12][5],
                    "rainfall" : nov_info[12][6],
                    "windspeed" : nov_info[12][9]
                },
                "december" :{
                    "probability" : dec_prob_list[12],
                    "avg_temp" : dec_info[12][1],
                    "clearer" : dec_info[12][4],
                    "precipitation" : dec_info[12][5],
                    "rainfall" : dec_info[12][6],
                    "windspeed" : dec_info[12][9]
                }
            }
        },
        {
            "destination" : "Shah Alam",
            "months" : {
                "january" :{
                    "probability" : jan_prob_list[13],
                    "avg_temp" : jan_info[13][1],
                    "clearer" : jan_info[13][4],
                    "precipitation" : jan_info[13][5],
                    "rainfall" : jan_info[13][6],
                    "windspeed" : jan_info[13][9]
                },
                "february" :{
                    "probability" : jan_prob_list[13],
                    "avg_temp" : feb_info[13][1],
                    "clearer" : feb_info[13][4],
                    "precipitation" : feb_info[13][5],
                    "rainfall" : feb_info[13][6],
                    "windspeed" : feb_info[13][9]
                },
                "march" :{
                    "probability" : mar_prob_list[13],
                    "avg_temp" : mar_info[13][1],
                    "clearer" : mar_info[13][4],
                    "precipitation" : mar_info[13][5],
                    "rainfall" : mar_info[13][6],
                    "windspeed" : mar_info[13][9]
                },
                "april" :{
                    "probability" : apr_prob_list[13],
                    "avg_temp" : apr_info[13][1],
                    "clearer" : apr_info[13][4],
                    "precipitation" : apr_info[13][5],
                    "rainfall" : apr_info[13][6],
                    "windspeed" : apr_info[13][9]
                },
                "may" :{
                    "probability" : may_prob_list[13],
                    "avg_temp" : may_info[13][1],
                    "clearer" : may_info[13][4],
                    "precipitation" : may_info[13][5],
                    "rainfall" : may_info[13][6],
                    "windspeed" : may_info[13][9]
                },
                "june" :{
                    "probability" : jun_prob_list[13],
                    "avg_temp" : jun_info[13][1],
                    "clearer" : jun_info[13][4],
                    "precipitation" : jun_info[13][5],
                    "rainfall" : jun_info[13][6],
                    "windspeed" : jun_info[13][9]
                },
                "july" :{
                    "probability" : jul_prob_list[13],
                    "avg_temp" : jul_info[13][1],
                    "clearer" : jul_info[13][4],
                    "precipitation" : jul_info[13][5],
                    "rainfall" : jul_info[13][6],
                    "windspeed" : jul_info[13][9]
                },
                "august" :{
                    "probability" : aug_prob_list[13],
                    "avg_temp" : aug_info[13][1],
                    "clearer" : aug_info[13][4],
                    "precipitation" : aug_info[13][5],
                    "rainfall" : aug_info[13][6],
                    "windspeed" : aug_info[13][9]
                },
                "september" :{
                    "probability" : sep_prob_list[13],
                    "avg_temp" : sep_info[13][1],
                    "clearer" : sep_info[13][4],
                    "precipitation" : sep_info[13][5],
                    "rainfall" : sep_info[13][6],
                    "windspeed" : sep_info[13][9]
                },
                "october" :{
                    "probability" : oct_prob_list[13],
                    "avg_temp" : oct_info[13][1],
                    "clearer" : oct_info[13][4],
                    "precipitation" : oct_info[13][5],
                    "rainfall" : oct_info[13][6],
                    "windspeed" : oct_info[13][9]
                },
                "november" :{
                    "probability" : nov_prob_list[13],
                    "avg_temp" : nov_info[13][1],
                    "clearer" : nov_info[13][4],
                    "precipitation" : nov_info[13][5],
                    "rainfall" : nov_info[13][6],
                    "windspeed" : nov_info[13][9]
                },
                "december" :{
                    "probability" : dec_prob_list[13],
                    "avg_temp" : dec_info[13][1],
                    "clearer" : dec_info[13][4],
                    "precipitation" : dec_info[13][5],
                    "rainfall" : dec_info[13][6],
                    "windspeed" : dec_info[13][9]
                }
            }
        },
        {
            "destination" : "Kuala Lumpur",
            "months" : {
                "january" :{
                    "probability" : jan_prob_list[14],
                    "avg_temp" : jan_info[14][1],
                    "clearer" : jan_info[14][4],
                    "precipitation" : jan_info[14][5],
                    "rainfall" : jan_info[14][6],
                    "windspeed" : jan_info[14][9]
                },
                "february" :{
                    "probability" : jan_prob_list[14],
                    "avg_temp" : feb_info[14][1],
                    "clearer" : feb_info[14][4],
                    "precipitation" : feb_info[14][5],
                    "rainfall" : feb_info[14][6],
                    "windspeed" : feb_info[14][9]
                },
                "march" :{
                    "probability" : mar_prob_list[14],
                    "avg_temp" : mar_info[14][1],
                    "clearer" : mar_info[14][4],
                    "precipitation" : mar_info[14][5],
                    "rainfall" : mar_info[14][6],
                    "windspeed" : mar_info[14][9]
                },
                "april" :{
                    "probability" : apr_prob_list[14],
                    "avg_temp" : apr_info[14][1],
                    "clearer" : apr_info[14][4],
                    "precipitation" : apr_info[14][5],
                    "rainfall" : apr_info[14][6],
                    "windspeed" : apr_info[14][9]
                },
                "may" :{
                    "probability" : may_prob_list[14],
                    "avg_temp" : may_info[14][1],
                    "clearer" : may_info[14][4],
                    "precipitation" : may_info[14][5],
                    "rainfall" : may_info[14][6],
                    "windspeed" : may_info[14][9]
                },
                "june" :{
                    "probability" : jun_prob_list[14],
                    "avg_temp" : jun_info[14][1],
                    "clearer" : jun_info[14][4],
                    "precipitation" : jun_info[14][5],
                    "rainfall" : jun_info[14][6],
                    "windspeed" : jun_info[14][9]
                },
                "july" :{
                    "probability" : jul_prob_list[14],
                    "avg_temp" : jul_info[14][1],
                    "clearer" : jul_info[14][4],
                    "precipitation" : jul_info[14][5],
                    "rainfall" : jul_info[14][6],
                    "windspeed" : jul_info[14][9]
                },
                "august" :{
                    "probability" : aug_prob_list[14],
                    "avg_temp" : aug_info[14][1],
                    "clearer" : aug_info[14][4],
                    "precipitation" : aug_info[14][5],
                    "rainfall" : aug_info[14][6],
                    "windspeed" : aug_info[14][9]
                },
                "september" :{
                    "probability" : sep_prob_list[14],
                    "avg_temp" : sep_info[14][1],
                    "clearer" : sep_info[14][4],
                    "precipitation" : sep_info[14][5],
                    "rainfall" : sep_info[14][6],
                    "windspeed" : sep_info[14][9]
                },
                "october" :{
                    "probability" : oct_prob_list[14],
                    "avg_temp" : oct_info[14][1],
                    "clearer" : oct_info[14][4],
                    "precipitation" : oct_info[14][5],
                    "rainfall" : oct_info[14][6],
                    "windspeed" : oct_info[14][9]
                },
                "november" :{
                    "probability" : nov_prob_list[14],
                    "avg_temp" : nov_info[14][1],
                    "clearer" : nov_info[14][4],
                    "precipitation" : nov_info[14][5],
                    "rainfall" : nov_info[14][6],
                    "windspeed" : nov_info[14][9]
                },
                "december" :{
                    "probability" : dec_prob_list[14],
                    "avg_temp" : dec_info[14][1],
                    "clearer" : dec_info[14][4],
                    "precipitation" : dec_info[14][5],
                    "rainfall" : dec_info[14][6],
                    "windspeed" : dec_info[14][9]
                }
            }
        },
        {
            "destination" : "Putrajaya",
            "months" : {
                "january" :{
                    "probability" : jan_prob_list[15],
                    "avg_temp" : jan_info[15][1],
                    "clearer" : jan_info[15][4],
                    "precipitation" : jan_info[15][5],
                    "rainfall" : jan_info[15][6],
                    "windspeed" : jan_info[15][9]
                },
                "february" :{
                    "probability" : jan_prob_list[15],
                    "avg_temp" : feb_info[15][1],
                    "clearer" : feb_info[15][4],
                    "precipitation" : feb_info[15][5],
                    "rainfall" : feb_info[15][6],
                    "windspeed" : feb_info[15][9]
                },
                "march" :{
                    "probability" : mar_prob_list[15],
                    "avg_temp" : mar_info[15][1],
                    "clearer" : mar_info[15][4],
                    "precipitation" : mar_info[15][5],
                    "rainfall" : mar_info[15][6],
                    "windspeed" : mar_info[15][9]
                },
                "april" :{
                    "probability" : apr_prob_list[15],
                    "avg_temp" : apr_info[15][1],
                    "clearer" : apr_info[15][4],
                    "precipitation" : apr_info[15][5],
                    "rainfall" : apr_info[15][6],
                    "windspeed" : apr_info[15][9]
                },
                "may" :{
                    "probability" : may_prob_list[15],
                    "avg_temp" : may_info[15][1],
                    "clearer" : may_info[15][4],
                    "precipitation" : may_info[15][5],
                    "rainfall" : may_info[15][6],
                    "windspeed" : may_info[15][9]
                },
                "june" :{
                    "probability" : jun_prob_list[15],
                    "avg_temp" : jun_info[15][1],
                    "clearer" : jun_info[15][4],
                    "precipitation" : jun_info[15][5],
                    "rainfall" : jun_info[15][6],
                    "windspeed" : jun_info[15][9]
                },
                "july" :{
                    "probability" : jul_prob_list[15],
                    "avg_temp" : jul_info[15][1],
                    "clearer" : jul_info[15][4],
                    "precipitation" : jul_info[15][5],
                    "rainfall" : jul_info[15][6],
                    "windspeed" : jul_info[15][9]
                },
                "august" :{
                    "probability" : aug_prob_list[15],
                    "avg_temp" : aug_info[15][1],
                    "clearer" : aug_info[15][4],
                    "precipitation" : aug_info[15][5],
                    "rainfall" : aug_info[15][6],
                    "windspeed" : aug_info[15][9]
                },
                "september" :{
                    "probability" : sep_prob_list[15],
                    "avg_temp" : sep_info[15][1],
                    "clearer" : sep_info[15][4],
                    "precipitation" : sep_info[15][5],
                    "rainfall" : sep_info[15][6],
                    "windspeed" : sep_info[15][9]
                },
                "october" :{
                    "probability" : oct_prob_list[15],
                    "avg_temp" : oct_info[15][1],
                    "clearer" : oct_info[15][4],
                    "precipitation" : oct_info[15][5],
                    "rainfall" : oct_info[15][6],
                    "windspeed" : oct_info[15][9]
                },
                "november" :{
                    "probability" : nov_prob_list[15],
                    "avg_temp" : nov_info[15][1],
                    "clearer" : nov_info[15][4],
                    "precipitation" : nov_info[15][5],
                    "rainfall" : nov_info[15][6],
                    "windspeed" : nov_info[15][9]
                },
                "december" :{
                    "probability" : dec_prob_list[15],
                    "avg_temp" : dec_info[15][1],
                    "clearer" : dec_info[15][4],
                    "precipitation" : dec_info[15][5],
                    "rainfall" : dec_info[15][6],
                    "windspeed" : dec_info[15][9]
                }
            }
        },
        {
            "destination" : "Seremban",
            "months" : {
                "january" :{
                    "probability" : jan_prob_list[16],
                    "avg_temp" : jan_info[16][1],
                    "clearer" : jan_info[16][4],
                    "precipitation" : jan_info[16][5],
                    "rainfall" : jan_info[16][6],
                    "windspeed" : jan_info[16][9]
                },
                "february" :{
                    "probability" : jan_prob_list[16],
                    "avg_temp" : feb_info[16][1],
                    "clearer" : feb_info[16][4],
                    "precipitation" : feb_info[16][5],
                    "rainfall" : feb_info[16][6],
                    "windspeed" : feb_info[16][9]
                },
                "march" :{
                    "probability" : mar_prob_list[16],
                    "avg_temp" : mar_info[16][1],
                    "clearer" : mar_info[16][4],
                    "precipitation" : mar_info[16][5],
                    "rainfall" : mar_info[16][6],
                    "windspeed" : mar_info[16][9]
                },
                "april" :{
                    "probability" : apr_prob_list[16],
                    "avg_temp" : apr_info[16][1],
                    "clearer" : apr_info[16][4],
                    "precipitation" : apr_info[16][5],
                    "rainfall" : apr_info[16][6],
                    "windspeed" : apr_info[16][9]
                },
                "may" :{
                    "probability" : may_prob_list[16],
                    "avg_temp" : may_info[16][1],
                    "clearer" : may_info[16][4],
                    "precipitation" : may_info[16][5],
                    "rainfall" : may_info[16][6],
                    "windspeed" : may_info[16][9]
                },
                "june" :{
                    "probability" : jun_prob_list[16],
                    "avg_temp" : jun_info[16][1],
                    "clearer" : jun_info[16][4],
                    "precipitation" : jun_info[16][5],
                    "rainfall" : jun_info[16][6],
                    "windspeed" : jun_info[16][9]
                },
                "july" :{
                    "probability" : jul_prob_list[16],
                    "avg_temp" : jul_info[16][1],
                    "clearer" : jul_info[16][4],
                    "precipitation" : jul_info[16][5],
                    "rainfall" : jul_info[16][6],
                    "windspeed" : jul_info[16][9]
                },
                "august" :{
                    "probability" : aug_prob_list[16],
                    "avg_temp" : aug_info[16][1],
                    "clearer" : aug_info[16][4],
                    "precipitation" : aug_info[16][5],
                    "rainfall" : aug_info[16][6],
                    "windspeed" : aug_info[16][9]
                },
                "september" :{
                    "probability" : sep_prob_list[16],
                    "avg_temp" : sep_info[16][1],
                    "clearer" : sep_info[16][4],
                    "precipitation" : sep_info[16][5],
                    "rainfall" : sep_info[16][6],
                    "windspeed" : sep_info[16][9]
                },
                "october" :{
                    "probability" : oct_prob_list[16],
                    "avg_temp" : oct_info[16][1],
                    "clearer" : oct_info[16][4],
                    "precipitation" : oct_info[16][5],
                    "rainfall" : oct_info[16][6],
                    "windspeed" : oct_info[16][9]
                },
                "november" :{
                    "probability" : nov_prob_list[16],
                    "avg_temp" : nov_info[16][1],
                    "clearer" : nov_info[16][4],
                    "precipitation" : nov_info[16][5],
                    "rainfall" : nov_info[16][6],
                    "windspeed" : nov_info[16][9]
                },
                "december" :{
                    "probability" : dec_prob_list[16],
                    "avg_temp" : dec_info[16][1],
                    "clearer" : dec_info[16][4],
                    "precipitation" : dec_info[16][5],
                    "rainfall" : dec_info[16][6],
                    "windspeed" : dec_info[16][9]
                }
            }
        },
        {
            "destination" : "Malacca",
            "months" : {
                "january" :{
                    "probability" : jan_prob_list[17],
                    "avg_temp" : jan_info[17][1],
                    "clearer" : jan_info[17][4],
                    "precipitation" : jan_info[17][5],
                    "rainfall" : jan_info[17][6],
                    "windspeed" : jan_info[17][9]
                },
                "february" :{
                    "probability" : jan_prob_list[17],
                    "avg_temp" : feb_info[17][1],
                    "clearer" : feb_info[17][4],
                    "precipitation" : feb_info[17][5],
                    "rainfall" : feb_info[17][6],
                    "windspeed" : feb_info[17][9]
                },
                "march" :{
                    "probability" : mar_prob_list[17],
                    "avg_temp" : mar_info[17][1],
                    "clearer" : mar_info[17][4],
                    "precipitation" : mar_info[17][5],
                    "rainfall" : mar_info[17][6],
                    "windspeed" : mar_info[17][9]
                },
                "april" :{
                    "probability" : apr_prob_list[17],
                    "avg_temp" : apr_info[17][1],
                    "clearer" : apr_info[17][4],
                    "precipitation" : apr_info[17][5],
                    "rainfall" : apr_info[17][6],
                    "windspeed" : apr_info[17][9]
                },
                "may" :{
                    "probability" : may_prob_list[17],
                    "avg_temp" : may_info[17][1],
                    "clearer" : may_info[17][4],
                    "precipitation" : may_info[17][5],
                    "rainfall" : may_info[17][6],
                    "windspeed" : may_info[17][9]
                },
                "june" :{
                    "probability" : jun_prob_list[17],
                    "avg_temp" : jun_info[17][1],
                    "clearer" : jun_info[17][4],
                    "precipitation" : jun_info[17][5],
                    "rainfall" : jun_info[17][6],
                    "windspeed" : jun_info[17][9]
                },
                "july" :{
                    "probability" : jul_prob_list[17],
                    "avg_temp" : jul_info[17][1],
                    "clearer" : jul_info[17][4],
                    "precipitation" : jul_info[17][5],
                    "rainfall" : jul_info[17][6],
                    "windspeed" : jul_info[17][9]
                },
                "august" :{
                    "probability" : aug_prob_list[17],
                    "avg_temp" : aug_info[17][1],
                    "clearer" : aug_info[17][4],
                    "precipitation" : aug_info[17][5],
                    "rainfall" : aug_info[17][6],
                    "windspeed" : aug_info[17][9]
                },
                "september" :{
                    "probability" : sep_prob_list[17],
                    "avg_temp" : sep_info[17][1],
                    "clearer" : sep_info[17][4],
                    "precipitation" : sep_info[17][5],
                    "rainfall" : sep_info[17][6],
                    "windspeed" : sep_info[17][9],
                },
                "october" :{
                    "probability" : oct_prob_list[17],
                    "avg_temp" : oct_info[17][1],
                    "clearer" : oct_info[17][4],
                    "precipitation" : oct_info[17][5],
                    "rainfall" : oct_info[17][6],
                    "windspeed" : oct_info[17][9]
                },
                "november" :{
                    "probability" : nov_prob_list[17],
                    "avg_temp" : nov_info[17][1],
                    "clearer" : nov_info[17][4],
                    "precipitation" : nov_info[17][5],
                    "rainfall" : nov_info[17][6],
                    "windspeed" : nov_info[17][9]
                },
                "december" :{
                    "probability" : dec_prob_list[17],
                    "avg_temp" : dec_info[17][1],
                    "clearer" : dec_info[17][4],
                    "precipitation" : dec_info[17][5],
                    "rainfall" : dec_info[17][6],
                    "windspeed" : dec_info[17][9]
                }
            }
        },
        {
            "destination" : "Johor Bahru",
            "months" : {
                "january" :{
                    "probability" : jan_prob_list[18],
                    "avg_temp" : jan_info[18][1],
                    "clearer" : jan_info[18][4],
                    "precipitation" : jan_info[18][5],
                    "rainfall" : jan_info[18][6],
                    "windspeed" : jan_info[18][9]
                },
                "february" :{
                    "probability" : jan_prob_list[18],
                    "avg_temp" : feb_info[18][1],
                    "clearer" : feb_info[18][4],
                    "precipitation" : feb_info[18][5],
                    "rainfall" : feb_info[18][6],
                    "windspeed" : feb_info[18][9]
                },
                "march" :{
                    "probability" : mar_prob_list[18],
                    "avg_temp" : mar_info[18][1],
                    "clearer" : mar_info[18][4],
                    "precipitation" : mar_info[18][5],
                    "rainfall" : mar_info[18][6],
                    "windspeed" : mar_info[18][9]
                },
                "april" :{
                    "probability" : apr_prob_list[18],
                    "avg_temp" : apr_info[18][1],
                    "clearer" : apr_info[18][4],
                    "precipitation" : apr_info[18][5],
                    "rainfall" : apr_info[18][6],
                    "windspeed" : apr_info[18][9]
                },
                "may" :{
                    "probability" : may_prob_list[18],
                    "avg_temp" : may_info[18][1],
                    "clearer" : may_info[18][4],
                    "precipitation" : may_info[18][5],
                    "rainfall" : may_info[18][6],
                    "windspeed" : may_info[18][9]
                },
                "june" :{
                    "probability" : jun_prob_list[18],
                    "avg_temp" : jun_info[18][1],
                    "clearer" : jun_info[18][4],
                    "precipitation" : jun_info[18][5],
                    "rainfall" : jun_info[18][6],
                    "windspeed" : jun_info[18][9]
                },
                "july" :{
                    "probability" : jul_prob_list[18],
                    "avg_temp" : jul_info[18][1],
                    "clearer" : jul_info[18][4],
                    "precipitation" : jul_info[18][5],
                    "rainfall" : jul_info[18][6],
                    "windspeed" : jul_info[18][9]
                },
                "august" :{
                    "probability" : aug_prob_list[18],
                    "avg_temp" : aug_info[18][1],
                    "clearer" : aug_info[18][4],
                    "precipitation" : aug_info[18][5],
                    "rainfall" : aug_info[18][6],
                    "windspeed" : aug_info[18][9]
                },
                "september" :{
                    "probability" : sep_prob_list[18],
                    "avg_temp" : sep_info[18][1],
                    "clearer" : sep_info[18][4],
                    "precipitation" : sep_info[18][5],
                    "rainfall" : sep_info[18][6],
                    "windspeed" : sep_info[18][9]
                },
                "october" :{
                    "probability" : oct_prob_list[18],
                    "avg_temp" : oct_info[18][1],
                    "clearer" : oct_info[18][4],
                    "precipitation" : oct_info[18][5],
                    "rainfall" : oct_info[18][6],
                    "windspeed" : oct_info[18][9]
                },
                "november" :{
                    "probability" : nov_prob_list[18],
                    "avg_temp" : nov_info[18][1],
                    "clearer" : nov_info[18][4],
                    "precipitation" : nov_info[18][5],
                    "rainfall" : nov_info[18][6],
                    "windspeed" : nov_info[18][9]
                },
                "december" :{
                    "probability" : dec_prob_list[18],
                    "avg_temp" : dec_info[18][1],
                    "clearer" : dec_info[18][4],
                    "precipitation" : dec_info[18][5],
                    "rainfall" : dec_info[18][6],
                    "windspeed" : dec_info[18][9]
                }
            }
        }
    ]
}

In [ ]:
# Open a JSON file and store the weather intents in the file
with open("weather_intents.json", "w") as outfile:
    json.dump(weather_intents, outfile)